In [8]:
# Import BeautifulSoup for parsing and splinter for site navigation
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import pandas as pd
import time
import pymongo

#executable_path = {"executable_path": "/Users/chr/Desktop/Working Directory/chromedriver"}
#browser = Browser("chrome", **executable_path, headless=False)
#combined browser path
browser = Browser('chrome', executable_path="chromedriver.exe", headless=False)

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

### NASA Mars News

In [9]:
# Visit the NASA news URL
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)

In [10]:
# Scrape page into soup
html = browser.html
soup = bs(html, 'html.parser')

In [11]:
#collect the latest News Title and Paragraph Text.
#Assign the text to variables to reference later
# Example variable:
#news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"
#news_title = "Beyond Mars, the Mini MarCO Spacecraft Fall Silent"

# save the most recent article, title and date
article = soup.find("div", class_="list_text")
news_p = article.find("div", class_="article_teaser_body").text
news_title = article.find("div", class_="content_title").text
news_date = article.find("div", class_="list_date").text
print(news_date)
print(news_title)
print(news_p)

February 13, 2019
Six Things to Know About NASA's Opportunity Rover
Opportunity's mission is complete. Here are highlights from its time on Mars.


### JPL Mars Space Images - Featured Image

In [17]:
# Visit JPL Featured Space Image url
images_url = "https://jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(images_url)

# Use splinter to navigate site and find the image url for the current Featured Mars Image
# assign the url string to a variable called `featured_image_url`.
# Scrape browser into soup and use soup to find the image of mars
# Save image url to a variable called `img_url`

html = browser.html
soup = bs(html, 'html.parser')
image = soup.find('li',class_='slide').a['data-fancybox-href']

img_url = "https://jpl.nasa.gov"+image
featured_image_url = img_url

print(featured_image_url)

https://jpl.nasa.gov/spaceimages/images/largesize/PIA22928_hires.jpg


In [ ]:
# Example:
#featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'

### Mars Weather

In [19]:
#* Visit the Mars Weather twitter account
#*Scrape the latest Mars weather tweet from the Twitter page
twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)

# Wait for page to load
time.sleep(2)

# Pass the HTML object from the splinter session
html = browser.html  
soup = bs(html, 'html.parser')

# Find all elements that contain tweets <div container
latest_tweets = soup.find_all('div', class_='js-tweet-text-container')

for tweet_html in latest_tweets:
    tweet = tweet_html.text.strip()
    if tweet.strip().startswith('Sol'): 
        mars_weather = tweet
        break
print(mars_weather)

Sol 2317 (2019-02-11), high -16C/3F, low -73C/-99F, pressure at 8.12 hPa, daylight 06:46-18:52pic.twitter.com/D15lfEs6RN


In [ ]:
# Example:
mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'

### Mars Facts

In [ ]:
#* 4a Visit the Mars Facts webpage
#use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
facts_url = "http://space-facts.com/mars/"
browser.visit(facts_url)

In [42]:
#* 4b Use Pandas to convert data to HTML table string
table = pd.read_html(facts_url)
type(table)

mars_df = table[0]
mars_df.columns=['Description','Value']
mars_df.set_index('Description', inplace=True)
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [30]:
#5a* Visit the USGS Astrogeology
#scrape high resolution images for each of Mar's hemispheres.
astr_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
base = "https://astrogeology.usgs.gov/"
browser.visit(astr_url)

# Use splinter to loop through the 4 images and load them into a dictionary
import time 
html = browser.html
soup = bs(html, 'html.parser')

In [31]:
#*5b You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
#*5c Save both the image url string for the full resolution hemisphere image, 
#*5d Save Hemisphere title containing the hemisphere name. 
#*5e Use a Python dictionary to store the data using the keys `img_url` and `title`.
#*5f Append the dictionary with the image url string and the hemisphere title to a list.
#*5g This list will contain one dictionary for each hemisphere.
# loop through the four tags and load the data to the dictionary

for i in range (4):
    time.sleep(5)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemis.append(dictionary)
    browser.back()

In [32]:
print(mars_hemis)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [ ]:
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]